# 추천시스템

In [23]:
import pandas as pd
import numpy as np

In [24]:
data = pd.read_csv('./final_data.csv', encoding='euc-kr')

In [25]:
data

,Unnamed: 0,No,원래이름,상품명,최종상품명,수량
0,0,1,댓골두부찌개용,두부,두부,2.0
1,1,1,댓골두부찌개용,두부,두부,2.0
2,2,2,종가집콩이가득두부찌개용,두부,두부,2.0
3,3,5,강릉심층수순두부,두부,두부,1.0
4,4,16,백설두루두루두부,두부,두부,1.0
...,...,...,...,...,...,...
3532,3532,963,건호박국내산,건호박국내산,호박,1.0
3533,3533,966,번개,번개,번개,2.0
3534,3534,971,돈나물,돈나물,나물,1.0
3535,3535,974,립톤아이스티레몬,립톤아이스티레몬,음료,1.0


In [30]:
ad= data['최종상품명']
len(ad)

3537

In [27]:
from collections import Counter

In [38]:
pp=Counter(ad)
ppp=pp.most_common(n=100)

In [12]:
from wordcloud import WordCloud

In [13]:
wc = WordCloud(font_path='C:/Users/USER/Desktop/NanumFontSetup_TTF_GOTHIC/NanumGothic.ttf',
              background_color='white',
              width=1000,
              height=1000,
              max_words=100,
              max_font_size=300)

In [39]:
wc.generate_from_frequencies(dict(ppp))
wc.to_file('wc.png')

In [31]:
df = data[['No','최종상품명','수량']]
df = df.drop(df.loc[df['최종상품명']=='소주공병'].index)

In [32]:
df.describe()

,No,수량
count,3526.000000,3526.000000
mean,479.427113,1.285309
std,271.287549,1.285487
min,1.000000,1.000000
25%,260.000000,1.000000
50%,464.000000,1.000000
75%,708.750000,1.000000
max,975.000000,27.000000


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3526 entries, 0 to 3536
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   No      3526 non-null   int64  
 1   최종상품명   3526 non-null   object 
 2   수량      3526 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 110.2+ KB


### surprise package

In [68]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import KNNBasic
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import KNNBaseline

In [36]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0,27))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['No','최종상품명','수량']], reader)

### cross_validate로 가장 좋은 알고리즘 고르기

In [44]:
# ALS
print('Using ALS')
bsl_options_1 = {'method': 'als'}
algo_1 = BaselineOnly(bsl_options=bsl_options_1)

Using ALS


In [46]:
cross_validate(algo_1, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9644  0.8862  1.1312  1.0812  1.4787  1.1084  0.2042  
MAE (testset)     0.4141  0.4106  0.4460  0.4144  0.4955  0.4361  0.0323  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([0.9643657 , 0.88623191, 1.13124605, 1.08120557, 1.4787298 ]),
 'test_mae': array([0.41407669, 0.41063419, 0.44602971, 0.41443609, 0.49546563]),
 'fit_time': (0.003274202346801758,
  0.0029914379119873047,
  0.003989696502685547,
  0.0025348663330078125,
  0.002503633499145508),
 'test_time': (0.001996278762817383,
  0.0015041828155517578,
  0.001501321792602539,
  0.0019593238830566406,
  0.001992940902709961)}

In [47]:
# SGD
print('Using SGD')
bsl_options_2 = {'method': 'sgd'}
algo_2 = BaselineOnly(bsl_options=bsl_options_2)

Using SGD


In [48]:
cross_validate(algo_2, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9110  1.3562  1.2990  1.1264  0.8632  1.1112  0.1985  
MAE (testset)     0.4288  0.4948  0.4691  0.3902  0.3938  0.4354  0.0412  
Fit time          0.01    0.01    0.01    0.01    0.01    0.01    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([0.91097502, 1.35624098, 1.29896771, 1.12638649, 0.86323851]),
 'test_mae': array([0.42876364, 0.49484494, 0.469109  , 0.39024131, 0.3938268 ]),
 'fit_time': (0.008080244064331055,
  0.007977962493896484,
  0.006981372833251953,
  0.006981372833251953,
  0.0070133209228515625),
 'test_time': (0.0019953250885009766,
  0.001994609832763672,
  0.001994609832763672,
  0.0019948482513427734,
  0.001994609832763672)}

In [54]:
# KNN
print('using KNN')
algo_3 = KNNBasic()

using KNN


In [55]:
cross_validate(algo_3, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8907  0.9864  1.3134  1.1875  1.6239  1.2004  0.2585  
MAE (testset)     0.3180  0.3580  0.4198  0.4377  0.4664  0.4000  0.0543  
Fit time          0.02    0.02    0.02    0.02    0.02    0.02    0.00    
Test time         0.03    0.03    0.03    0.04    0.04    0.03    0.00    


{'test_rmse': array([0.8907277 , 0.98640967, 1.31336189, 1.1874664 , 1.6239286 ]),
 'test_mae': array([0.318001  , 0.35795006, 0.41981228, 0.43768554, 0.46641419]),
 'fit_time': (0.018991708755493164,
  0.018003225326538086,
  0.01695537567138672,
  0.016954660415649414,
  0.018949031829833984),
 'test_time': (0.03453421592712402,
  0.03493785858154297,
  0.032911062240600586,
  0.0359039306640625,
  0.0359036922454834)}

## parameter 튜닝하기

In [75]:
param_grid = {'bsl_options': {'method': ['als', 'sgd'],
                              'reg': [1, 2]},'k': [2, 3, 4, 5]
              }

gs = GridSearchCV(KNNBaseline, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

In [76]:
# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.2993767472509263
{'bsl_options': {'method': 'als', 'reg': 1}, 'k': 5}


In [77]:
results_df = pd.DataFrame.from_dict(gs.cv_results)
results_df

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_bsl_options,param_k
0,1.248343,1.490826,1.326975,1.355381,0.101010,13,0.416935,0.482576,0.440727,0.446746,0.027134,11,0.016296,0.000465,0.034929,0.000845,"{'bsl_options': {'method': 'als', 'reg': 1}, '...","{'method': 'als', 'reg': 1}",2
1,1.219573,1.436993,1.275752,1.310772,0.092151,5,0.408463,0.475453,0.429500,0.437805,0.027972,5,0.015498,0.000409,0.036686,0.000291,"{'bsl_options': {'method': 'als', 'reg': 1}, '...","{'method': 'als', 'reg': 1}",3
2,1.220797,1.437914,1.248007,1.302239,0.096578,3,0.410467,0.478935,0.423211,0.437538,0.029731,3,0.016364,0.000572,0.037462,0.001363,"{'bsl_options': {'method': 'als', 'reg': 1}, '...","{'method': 'als', 'reg': 1}",4
3,1.220467,1.435573,1.242091,1.299377,0.096709,1,0.409729,0.477263,0.416287,0.434426,0.030408,1,0.015795,0.000438,0.038035,0.000671,"{'bsl_options': {'method': 'als', 'reg': 1}, '...","{'method': 'als', 'reg': 1}",5
4,1.248343,1.490826,1.326975,1.355381,0.101010,14,0.416935,0.482576,0.440727,0.446746,0.027134,12,0.015470,0.000401,0.034581,0.000476,"{'bsl_options': {'method': 'als', 'reg': 2}, '...","{'method': 'als', 'reg': 2}",2
5,1.219573,1.436993,1.275752,1.310772,0.092151,6,0.408463,0.475453,0.429500,0.437805,0.027972,6,0.016161,0.000614,0.035929,0.000749,"{'bsl_options': {'method': 'als', 'reg': 2}, '...","{'method': 'als', 'reg': 2}",3
6,1.220797,1.437914,1.248007,1.302239,0.096578,4,0.410467,0.478935,0.423211,0.437538,0.029731,4,0.016446,0.001233,0.038909,0.001422,"{'bsl_options': {'method': 'als', 'reg': 2}, '...","{'method': 'als', 'reg': 2}",4
7,1.220467,1.435573,1.242091,1.299377,0.096709,2,0.409729,0.477263,0.416287,0.434426,0.030408,2,0.015936,0.000824,0.037961,0.000421,"{'bsl_options': {'method': 'als', 'reg': 2}, '...","{'method': 'als', 'reg': 2}",5
8,1.250504,1.517474,1.344272,1.370750,0.110587,15,0.421812,0.494730,0.452548,0.456363,0.029891,16,0.019739,0.000662,0.034908,0.000789,"{'bsl_options': {'method': 'sgd', 'reg': 1}, '...","{'method': 'sgd', 'reg': 1}",2
9,1.222466,1.467516,1.294944,1.328309,0.102785,9,0.412437,0.489191,0.440223,0.447284,0.031730,13,0.019897,0.000069,0.035916,0.000805,"{'bsl_options': {'method': 'sgd', 'reg': 1}, '...","{'method': 'sgd', 'reg': 1}",3


In [79]:
trainset, testset = train_test_split(data, test_size=.25)

In [80]:
algo = gs.best_estimator['rmse']
algo.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


## 특정 유저에 대해 추천하는 process

In [85]:
item_name = df['최종상품명'].unique()

In [97]:
# user1에 대해 상품 추천
uid=1
    
buy_item = df[df['No']==uid]['최종상품명'].unique()
total_items = df['최종상품명'].unique()

unbuy_item = [item for item in total_items if item not in buy_item]

print(uid, '번째 유저가 산 물건의 갯수는 ',len(buy_item),'개 입니다.')

1 번째 유저가 평점을 내린 노래는  6 개 입니다.


In [108]:
# user가 사지 않은 상품에 대해 살 갯수 예측
pred={}

for item in unbuy_item:

    recom = algo.predict(uid,item)
    pred[recom[1]]=recom[3]


In [109]:
pred_sorted = sorted(pred.items(),
                    reverse=True,
                    key=lambda item:item[1])

In [112]:
# 상품추천
for key, value in pred_sorted[:10]:
    print(key,'를 ',value,'개 살 것으로 예상됩니다.')

음식물용기 를  19.024914774721168 개 살 것으로 예상됩니다.
물 를  6.381479692214732 개 살 것으로 예상됩니다.
식품 를  5.243953088459811 개 살 것으로 예상됩니다.
속옷 를  4.422806698022963 개 살 것으로 예상됩니다.
귤 를  3.173309571337812 개 살 것으로 예상됩니다.
참치 를  2.7846524359282867 개 살 것으로 예상됩니다.
비트 를  2.7355514809268993 개 살 것으로 예상됩니다.
굴 를  2.6081415462868778 개 살 것으로 예상됩니다.
누룽지 를  2.5357365305955932 개 살 것으로 예상됩니다.
스티커 를  2.508117562623645 개 살 것으로 예상됩니다.
